<a href="https://colab.research.google.com/github/rpfilomeno/JupiterNotebooks/blob/main/Whisper_STT_and_Coqui_TTS_VIDEO%3Dmodified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fine tune a VITS model with the Coqui TTS framework using audio samples of your choice.**

Thank you to all of the [Coqui TTS](https://https://github.com/coqui-ai/TTS) contributors, the OpenAI team,and [@Thorsten-Voice](https://https://www.youtube.com/c/ThorstenMueller ) (I think he wrote this original script). I have no idea who wrote the rnnoise loop, but they are great, too. I just smash things together. [-nn](https://https://www.youtube.com/c/NanoNomad )

If you have run the script before, and want to examine your training session using Tensorboard, set your dataset directories, then skip to the bottom section to load Tensorboard.

Run this cell to connect your Google Drive account to save files.

In [2]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Set dataset and trainer output directory (model states saved here, subdirectory of dataset). Set upload_dir and place all audio files (Wav, mp3) in upload directory which will be /content/drive/MyDrive/{upload_dir}

In [3]:
dataset_name = "me2-dataset" #@param {type:"string"}
output_directory = "traineroutput" #@param {type:"string"}
upload_dir = "mp3uploads" #@param {type:"string"}
upload_dir = "/content/drive/MyDrive/" + upload_dir
!mkdir $upload_dir

mkdir: cannot create directory ‘/content/drive/MyDrive/mp3uploads’: File exists


Cleanup previous audio processing

In [4]:
!rm -rf /content/drive/MyDrive/$upload_dir/out
!rm -rf /content/drive/MyDrive/$dataset_name/converted

Download and Build Rnnoise (https://github.com/xiph/rnnoise) and Requirements (Optional if not processing .wav files for a new dataset. Multiple passes of rnnoise over samples may degrade audio quality. You should also preview your output dataset before training to ensure that the processing has not degraded any of your samples.  Occasionally, it will denoise too much of the tail end of phrases)

In [5]:
!pip install pyloudnorm
!git clone https://github.com/xiph/rnnoise.git
!sudo apt-get install curl autoconf automake libtool python-dev pkg-config sox ffmpeg
%cd /content/rnnoise
!sh autogen.sh
!sh configure
!make clean
!make

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'rnnoise' already exists and is not an empty directory.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'python-dev-is-python2' instead of 'python-dev'
autoconf is already the newest version (2.69-11.1).
automake is already the newest version (1:1.16.1-4ubuntu6).
libtool is already the newest version (2.4.6-14).
pkg-config is already the newest version (0.29.1-0ubuntu4).
python-dev-is-python2 is already the newest version (2.7.17-4).
curl is already the newest version (7.68.0-1ubuntu2.16).
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
sox is already the newest version (14.4.2+git20190427-2+deb11u1build0.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
/content/rnnoise
Updating build configuration files for rnnoise, please wait....
libtoolize: putting auxiliary file

Install Sox, Install OpenAI Whisper STT+Translation (https://github.com/openai/whisper)

In [6]:
%cd /content
!sudo apt install sox
!git clone https://github.com/openai/whisper.git
!pip install git+https://github.com/openai/whisper.git 


/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
sox is already the newest version (14.4.2+git20190427-2+deb11u1build0.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
fatal: destination path 'whisper' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-1lgz3tjg
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-1lgz3tjg
  Resolved https://github.com/openai/whisper.git to commit 3e1780fd37686666f568be9c99f5b5e3e4f2eb92
  Preparing metadata (setup.py) ... done


Install Coqui TTS (https://github.com/coqui-ai/TTS), espeak-ng phonemeizer (https://github.com/espeak-ng/espeak-ng), download Coqui TTS source and examples from GitHub

In [7]:
#@title
%cd /content
!sudo apt-get install espeak-ng
!git clone https://github.com/coqui-ai/TTS.git
!pip install TTS
!tts --list_models

/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
espeak-ng is already the newest version (1.50+dfsg-6).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
fatal: destination path 'TTS' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
 Name format: type/language/dataset/model
 1: tts_models/multilingual/multi-dataset/your_tts
 2: tts_models/bg/cv/vits
 3: tts_models/cs/cv/vits
 4: tts_models/da/cv/vits
 5: tts_models/et/cv/vits
 6: tts_models/ga/cv/vits
 7: tts_models/en/ek1/tacotron2
 8: tts_models/en/ljspeech/tacotron2-DDC
 9: tts_models/en/ljspeech/tacotron2-DDC_ph
 10: tts_models/en/ljspeech/glow-tts
 11: tts_models/en/ljspeech/speedy-speech
 12: tts_models/en/ljspeech/tacotron2-DCA
 13: tts_models/en/ljspeech/vits [already downloaded]
 14: tts_models/en/ljspeech/vits--neon
 15: tts_models/en/ljspeech/fast_pitch
 16: tts

Convert mp3 files in upload_dir to wav as 22050hz mono wav
Place output wav files in directory named 'out'.
Copy any uploaded .wav files to 'out' as 22050hz mono wav.

Make directory 'splits'. Use sox to split wav files into 8 second segments, placed in 'splits'.

In [8]:
#@title
%cd $upload_dir
!rm -rf $upload_dir/out
!mkdir $upload_dir/out
!find . -name '*.mp3' -exec bash -c 'for f; do ffmpeg -i "$f" -acodec pcm_s16le -ar 22050 -ac 1 out/"${f%.mp3}".wav ; done' _ {} +
!find . -name '*.wav' -exec bash -c 'for f; do ffmpeg -i "$f" -acodec pcm_s16le -ar 22050 -ac 1 out/"${f%.wav}".wav ; done' _ {} +


!ls -al $upload_dir/out
!mkdir $upload_dir/out/splits
%cd $upload_dir/out
!for FILE in *.wav; do sox "$FILE" splits/"$FILE" --show-progress silence 1 0.5 0.1% 1 0.5 0.1% : newfile : restart ; done
#alt split method: force splits of 8 seconds, however this will split words. Comment the above with # and remove the # below to change
#!for FILE in *.wav; do sox "$FILE" splits/"$FILE" --show-progress trim 0 8 : restart ; done
%cd $upload_dir/out/splits
!find . -name "*.wav" -type f -size -15k -delete

/content/drive/MyDrive/mp3uploads
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable

(Optional) Run rnnoise, processed clips will be saved to your Google Drive in a folder named 'converted'. Files converted to 22khz mono for VITS model fine tuning.

Original author unknown, but thank you for posting this helpful code.

In [9]:
#@title
orig_wavs= upload_dir + "/out/splits"
from pathlib import Path
import os
import subprocess
import soundfile as sf
import pyloudnorm as pyln
import sys
import glob
rnn = "/content/rnnoise/examples/rnnoise_demo"
paths = glob.glob(os.path.join(orig_wavs, '*.wav'))
for filepath in paths:
  base = os.path.basename(filepath)
  tp_s = "/content/drive/MyDrive/" + dataset_name + "/" + "converted" + "/"
  tf_s = "/content/drive/MyDrive/" + dataset_name + "/" + "converted" + "/" + base
  target_path = Path(tp_s)
  target_file = Path(tf_s)
  print("From: " + str(filepath))
  print("To: " + str(target_file))
	
# Stereo to Mono; upsample to 48000Hz
# added -G to fix gain, -v 0.95
  subprocess.run(["sox", "-G", "-v", "0.95", filepath, "48k.wav", "remix", "-", "rate", "48000"])
  subprocess.run(["sox", "48k.wav", "-c", "1", "-r", "48000", "-b", "16", "-e", "signed-integer", "-t", "raw", "temp.raw"]) # convert wav to raw
  subprocess.run(["/content/rnnoise/examples/rnnoise_demo", "temp.raw", "rnn.raw"]) # apply rnnoise
  subprocess.run(["sox", "-G", "-v", "0.95", "-r", "48k", "-b", "16", "-e", "signed-integer", "rnn.raw", "-t", "wav", "rnn.wav"]) # convert raw back to wav

  subprocess.run(["mkdir", "-p", str(target_path)])
  subprocess.run(["sox", "rnn.wav", str(target_file), "remix", "-", "highpass", "100", "lowpass", "7000", "rate", "22050"]) # apply high/low pass filter and change sr to 22050Hz
  data, rate = sf.read(target_file)

# peak normalize audio to -1 dB
  peak_normalized_audio = pyln.normalize.peak(data, -1.0)

# measure the loudness first
  meter = pyln.Meter(rate) # create BS.1770 meter
  loudness = meter.integrated_loudness(data)

# loudness normalize audio to -25 dB LUFS
  loudness_normalized_audio = pyln.normalize.loudness(data, loudness, -25.0)
  sf.write(target_file, data=loudness_normalized_audio, samplerate=22050)
  print("")
%cd /content/drive/MyDrive/"$dataset_name"/converted/
!find . -name "*.wav" -type f -size -15k -delete


From: /content/drive/MyDrive/mp3uploads/out/splits/a1001.wav
To: /content/drive/MyDrive/me2-dataset/converted/a1001.wav

From: /content/drive/MyDrive/mp3uploads/out/splits/a2001.wav
To: /content/drive/MyDrive/me2-dataset/converted/a2001.wav

From: /content/drive/MyDrive/mp3uploads/out/splits/a3001.wav
To: /content/drive/MyDrive/me2-dataset/converted/a3001.wav

From: /content/drive/MyDrive/mp3uploads/out/splits/a4001.wav
To: /content/drive/MyDrive/me2-dataset/converted/a4001.wav

From: /content/drive/MyDrive/mp3uploads/out/splits/a5001.wav
To: /content/drive/MyDrive/me2-dataset/converted/a5001.wav

From: /content/drive/MyDrive/mp3uploads/out/splits/a6001.wav
To: /content/drive/MyDrive/me2-dataset/converted/a6001.wav

From: /content/drive/MyDrive/mp3uploads/out/splits/a7001.wav
To: /content/drive/MyDrive/me2-dataset/converted/a7001.wav

From: /content/drive/MyDrive/mp3uploads/out/splits/a8001.wav
To: /content/drive/MyDrive/me2-dataset/converted/a8001.wav

From: /content/drive/MyDrive/mp3

Copy wav files to dataset wav directory

In [10]:
#@title
!mkdir /content/drive/MyDrive/$dataset_name/wavs
!cp /content/drive/MyDrive/$dataset_name/converted/*.wav /content/drive/MyDrive/$dataset_name/wavs
!ls -al /content/drive/MyDrive/$dataset_name/wavs

mkdir: cannot create directory ‘/content/drive/MyDrive/me2-dataset/wavs’: File exists
total 15585
-rw------- 1 root root 2573720 Mar  5 15:35 a1001.wav
-rw------- 1 root root 2623994 Mar  5 15:35 a2001.wav
-rw------- 1 root root 2740418 Mar  5 15:35 a3001.wav
-rw------- 1 root root 1680254 Mar  5 15:35 a4001.wav
-rw------- 1 root root 2661478 Mar  5 15:35 a5001.wav
-rw------- 1 root root  514250 Mar  5 15:35 a6001.wav
-rw------- 1 root root  722842 Mar  5 15:35 a7001.wav
-rw------- 1 root root 1203974 Mar  5 15:35 a8001.wav
-rw------- 1 root root 1236168 Mar  5 15:35 a9001.wav


Run Whisper on generated audio clips, generate transcript named metadata.csv in LJSpeech format in the dataset directory.

In [11]:
#@title
import whisper
import os, os.path
import glob
import pandas as pd

from pathlib import Path

wavs = '/content/drive/MyDrive/'+dataset_name+'/wavs'

model = whisper.load_model("medium.en")
paths = glob.glob(os.path.join(wavs, '*.wav'))
print(len(paths))

all_filenames = []
transcript_text = []
with open('/content/drive/MyDrive/'+dataset_name+'/metadata.csv', 'w', encoding='utf-8') as outfile:
	for filepath in paths:
		base = os.path.basename(filepath)
		all_filenames.append(base)
	for filepath in paths:
		result = model.transcribe(filepath)
		output = result["text"].lstrip()
		output = output.replace("\n","")
		thefile = str(os.path.basename(filepath).lstrip(".")).rsplit(".")[0]
		outfile.write(thefile + '|' + output + '|' + output + '\n')
		print(thefile + '|' + output + '|' + output + '\n')


9
a1001|I have no sleep. No sleep! No sleep! Anyway. Bottom teeth revealed. No! No no no no no. I'm not gonna keep you that long for this stream. I don't think it's gonna be like the full hour long cuz I don't- I mean unless we get talking about something but otherwise I think this will be like a little bit extreme. What dances are you practicing? I can't tell you if I tell you not to kill you but I can't tell you. Oh but I can tell you that it's from quite a few. I need- ha! I need more time! I need more time! I only have like- what is it? Two days? Two days? Two days to learn? What is it? It was two days to learn. Uh... three? Three dances? Not enough time for me. I need more time. I need more time. Hopefully get more time.|I have no sleep. No sleep! No sleep! Anyway. Bottom teeth revealed. No! No no no no no. I'm not gonna keep you that long for this stream. I don't think it's gonna be like the full hour long cuz I don't- I mean unless we get talking about something but otherwise I 

Display dataset

In [18]:
#@title
!cat /content/drive/MyDrive/$dataset_name/metadata.csv

gura001|I have no sleep. No sleep! No sleep! Anyway. Bottom teeth revealed. No! No no no no no. I'm not gonna keep you that long for this stream. I don't think it's gonna be like the full hour long cuz I don't- I mean unless we get talking about something but otherwise I think this will be like a little bobby stream. What dances are you practicing? I can't tell you if I tell you not to kill you but I can't tell you. Oh but I can tell you that it's from quite a few. I need- ha! I need more time! I need more time! I only have like- what is it? Two days? Two days? Two days to learn? What is it? It was two days to learn. Uh three? Three dances? Not enough time for me. I need more time. I need more time. Hopefully get more time. Did you party too hard yesterday? Me? No! No I was trapped in a- I was trapped in the recording studio all day. I was trapped in the recording studio all day. It was kinda like a party though with all the snacks that the staff brought. I told members about this but 

Download VITS model and Generate Sample Wav File to /content/ljspeech-vits.wav  This will be deleted when your Colab session is closed.

In [12]:
!tts --text "I am the very model of a modern Major General" --model_name "tts_models/en/ljspeech/vits" --out_path /content/ljspeech-vits.wav

 > tts_models/en/ljspeech/vits is already downloaded.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Text: I am the very model of a modern Major General
 > Text splitted to sentences.
['I am the very model of a modern Major General']
 > Processing time: 5.6706554889678955
 > Re

Save the training script with your dataset name, and output dir set. File will be saved in your Google drive dataset folder as train_vits.py

In [27]:
#@title
code = """import os

from trainer import Trainer, TrainerArgs

from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import Vits, VitsAudioConfig
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor


os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

#output_path = os.path.dirname(os.path.abspath(__file__))
##########################################
#Change this to your dataset directory
##########################################
output_path = "/content/drive/MyDrive/"""
code = code + dataset_name + "/" + output_directory + "/" + "\""

code=code + """
dataset_config = BaseDatasetConfig(
##########################################
#Change this to your dataset directory
##########################################
    formatter="ljspeech", meta_file_train="metadata.csv", path=os.path.join(output_path, "/content/drive/MyDrive/"""
code = code + dataset_name
code=code + """")

)
audio_config = VitsAudioConfig(
    sample_rate=22050, win_length=1024, hop_length=256, num_mels=80, mel_fmin=0, mel_fmax=None
)

config = VitsConfig(
    audio=audio_config,
    run_name="vits_ljspeech",
    batch_size=4,
    eval_batch_size=4,
    batch_group_size=2,
#    num_loader_workers=8,
    num_loader_workers=1,
    num_eval_loader_workers=1,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=1000,
    save_step=1000,
	  save_checkpoints=True,
	  save_n_checkpoints=4,
	  save_best_after=1000,
    #text_cleaner="english_cleaners",
    text_cleaner="multilingual_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=True,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    cudnn_benchmark=False,

)

# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# config is updated with the default characters if not defined in the config.
tokenizer, config = TTSTokenizer.init_from_config(config)

# LOAD DATA SAMPLES
# Each sample is a list of ```[text, audio_file_path, speaker_name]```
# You can define your custom sample loader returning the list of samples.
# Or define your custom formatter and pass it to the `load_tts_samples`.
# Check `TTS.tts.datasets.load_tts_samples` for more details.
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=0.1111111111111111,
)

# init model
model = Vits(config, ap, tokenizer, speaker_manager=None)

# init the trainer and begin
trainer = Trainer(
    TrainerArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
  
)
trainer.fit()
"""
#print(code)
myFile = open("/content/drive/MyDrive/" + dataset_name + "/train_vits.py", 'w')

myFile.write(code)
myFile.close()
%ls -al /content/drive/MyDrive/$dataset_name/train_vits.py

-rw------- 1 root root 3057 Mar  5 15:50 /content/drive/MyDrive/me2-dataset/train_vits.py


Fine Tune VITS model

In [14]:
import torch
torch.cuda.empty_cache()
import gc
gc.collect()

17

In [15]:
!CUDA_VISIBLE_DEVICES="0" python /content/drive/MyDrive/$dataset_name/train_vits.py --restore_path /root/.local/share/tts/tts_models--en--ljspeech--vits/model_file.pth --config_file /root/.local/share/tts/tts_models--en--ljspeech--vits/config.json

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 | > Found 9 files in /content/drive/MyDrive/me2-dataset
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mou

**Resume a fine tuning session**

Run the next cell to list all of the saved run direcories.  Copy select the run you want to resume, ctrl+C to copy the name. Paste in the next cell and run it.

In [19]:
!ls /content/drive/MyDrive/$dataset_name/$output_directory

phoneme_cache  vits_ljspeech-March-05-2023_03+38PM-0000000


In [20]:
run_name = "vits_ljspeech-March-05-2023_03+38PM-0000000" #@param {type:"string"}

Run this cell to list all of the checkpoints in the saved run, then put the checkpoint name in the next cell, and run it.

In [21]:
!ls /content/drive/MyDrive/$dataset_name/$output_directory/$run_name

best_model_1000021.pth	events.out.tfevents.1678030690.3fa06e6e8b22
best_model.pth		trainer_0_log.txt
config.json		train_vits.py


In [22]:
model_checkpoint = "best_model_1000021.pth" #@param {type:"string"}

Run the next cell to restore a previous fine tuning session using your dataset name, trainer output directory, and model checkpoint set

In [ ]:
!CUDA_VISIBLE_DEVICES="0" python /content/drive/MyDrive/$dataset_name/train_vits.py --restore_path /content/drive/MyDrive/$dataset_name/$output_directory/$run_name/$model_checkpoint --config_path /content/drive/MyDrive/$dataset_name/$run_name/config.json

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 | > Found 9 files in /content/drive/MyDrive/me2-dataset
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mou

View Memory Usage

In [24]:
!nvidia-smi

Sun Mar  5 15:49:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    30W /  70W |   4227MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Load Tensorboard

In [25]:
import torch 
%load_ext tensorboard

Load Dashboard. May take several minutes to appear from a blank white box.  Ad blockers probably need to whitelist a bunch of Colab stuff or this won't wrok.

In [26]:
%tensorboard --logdir /content/drive/MyDrive/$dataset_name/$output_directory/

<IPython.core.display.Javascript object>